In [ ]:
node_count =           100
ant_count =            100
alpha =                .5
beta =                 1.2
evaporation =          .40
pheromone_constant =   1000.0
iterations =           10000

In [ ]:
import math
import datetime
import json

In [ ]:
with open('data/geocaches.json', 'r') as infile:
    data = json.load(infile)

In [ ]:
geocaches = data['geocaches'][:node_count]
nodes_latlon = data['locations_latlon'][:node_count]
nodes_lonlat = data['locations_lonlat'][:node_count]

nodes = dict([[index, coords] for index, coords in enumerate(nodes_latlon)])

In [ ]:
with open('data/matrix500.json', 'r') as infile:
    matrix_data = json.load(infile)

time_matrix = matrix_data['durations']

In [ ]:
for (i, matrix_row) in enumerate(time_matrix):
    for (j, value) in enumerate(matrix_row):
        if value == 0.0:
            time_matrix[i][j] = 1.0

In [ ]:
from ant_colony import ant_colony

def cost_function(from_index, to_index):
    return time_matrix[from_index][to_index]

# ...we can make a colony of ants...
colony = ant_colony(nodes, cost_function,
    ant_count=ant_count,
    alpha=alpha,
    beta=beta,
    pheromone_evaporation_coefficient=evaporation,
    pheromone_constant=pheromone_constant,
    iterations=iterations
)

# ...that will find the optimal solution with ACO
answer = colony.mainloop()

optimal_path = answer[0]
lowest_cost = answer[1]

In [ ]:
print(optimal_path)
print('\nLowest cost: ', math.floor(lowest_cost), f' ({str(datetime.timedelta(seconds=math.floor(lowest_cost)))})')

In [ ]:
from context import openroute

openroute.api.go_offline()
local_ready = openroute.check_status()
print("[{}] localhost".format(' OK ' if local_ready else 'FAIL'))

openroute.api.go_online()
azure_ready = openroute.check_status()
print("[{}] azure".format(' OK ' if azure_ready else 'FAIL'))

openroute.api.toggle_online(not local_ready)

print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))

In [ ]:
import folium
from algo_results import create_marker, create_circle_marker, fit_locations, style_function, create_legend

In [ ]:
map = folium.Map(tiles=None, location=nodes_latlon[optimal_path[0]], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(map)
pass

In [ ]:
max_geocaches = int(math.ceil((max(optimal_path) + 1) / 10.0)) * 10

In [ ]:
optimal_locations = [nodes_lonlat[id] for id in optimal_path]

step = 10
max_steps = math.ceil(len(optimal_locations) / step)

In [ ]:
for index in tqdm(range(max_steps)):
    start = index * step
    request = {
        'coordinates': optimal_locations[start:start+step+1],
        'geometry': True,
        'instructions': False
    }
    optimal_route = openroute.request_directions(request, format_out='/geojson')

    if optimal_route:
        folium.features.GeoJson(data=optimal_route,
                                name='Optimal route',
                                style_function=style_function('red'),
                                overlay=True).add_to(map)

fit_locations(map, nodes_latlon[:max_geocaches])

In [ ]:
for index, coords in enumerate(nodes_latlon[:max_geocaches]):
    color = '#787c99'
    if index == path[0]: # start
        color = '#41a6b5'
    elif index == path[-2]: # last node before start
        color = '#9d7cd8'
    elif index in path: # other visited
        color = '#3c7da2'
    create_marker(optimal_path, coords, index, color, radius=8).add_to(map)

In [ ]:
map